In [1]:
from resnet_variant import *
from baseline_cnn import *
from resnet_variant import resnet_n
import torch.nn as nn
import torch.optim as optim
import time
import pathlib
import torch
from evaluation import Evaluation
from xray_imbalanced_dataloader import create_balanced_split_loaders
import random
%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

class channelCopy(object):    
    def __call__(self, img):
        return torch.cat([img, img, img], 0)
    
def augmentImg(img): 
    if 0.9 < random.random():
        return img

    t=transforms.RandomRotation(10)
    img = t(img)
    return img
        
#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),augmentImg,transforms.ToTensor(),channelCopy()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)
# train_loader, val_loader, test_loader = create_balanced_split_loaders(batch_size, seed, transform=transform,
#                                                                          p_val=p_val, p_test=p_test,
#                                                                          shuffle=True, show_sample=False,
#                                                                          extras=extras, z_score=True)
# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = resnet_n(pretrained=False, num_classes=14)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = nn.MultiLabelSoftMarginLoss() #TODO - loss criteria are defined in the torch.nn package
criterion = nn.MultiLabelSoftMarginLoss()
#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters()) #TODO - optimizers are defined in the torch.optim package

CUDA NOT supported
Model on CUDA? False


In [12]:
transform = transforms.Compose([transforms.Resize(512), transforms.ToTensor()])

In [13]:
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

In [ ]:
for minibatch_count, (images, labels) in enumerate(train_loader, 0):
    

In [4]:
inbalance_weight = torch.Tensor([1/8236,1/1989,1/9561,1/14240,1/4175,1/4537,1/962,1/3825,1/3317,1/1637,1/1819,1/1237,1/2431,1/162])

In [5]:
inbalance_weight

tensor([1.2142e-04, 5.0277e-04, 1.0459e-04, 7.0225e-05, 2.3952e-04, 2.2041e-04,
        1.0395e-03, 2.6144e-04, 3.0148e-04, 6.1087e-04, 5.4975e-04, 8.0841e-04,
        4.1135e-04, 6.1728e-03])

In [10]:
inbalance_weight/torch.norm(inbalance_weight)

tensor([0.0189, 0.0783, 0.0163, 0.0109, 0.0373, 0.0343, 0.1619, 0.0407, 0.0470,
        0.0951, 0.0856, 0.1259, 0.0641, 0.9613])

In [8]:
F.normalize(inbalance_weight)

RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [2]:
x = torch.Tensor(2,3)

In [ ]:
{'Atelectasis': 8236, 'Effusion': 9561, 'Infiltration': 14240, 'Mass': 4175, 'Nodule': 4537, 'No Finding': 43579, 'Consolidation': 3317, 'Pneumothorax': 3825, 'Pleural_Thickening': 2431, 'Emphysema': 1819, 'Cardiomegaly': 1989, 'Pneumonia': 962, 'Fibrosis': 1237, 'Edema': 1637, 'Hernia': 162}

In [ ]:
{0: "Atelectasis", 1: "Cardiomegaly", 2: "Effusion",
                        3: "Infiltration", 4: "Mass", 5: "Nodule", 6: "Pneumonia",
                        7: "Pneumothorax", 8: "Consolidation", 9: "Edema",
                        10: "Emphysema", 11: "Fibrosis",
                        12: "Pleural_Thickening", 13: "Hernia"}

In [3]:
x

tensor([[-3.1832e+32,  4.5616e-41,  4.1694e-22],
        [ 3.0639e-41,  4.4842e-44,  0.0000e+00]])

In [4]:
x = torch.rand(2,3)

In [28]:
Path = '/datasets/ChestXray-NIHCC/images'
paths = []
images = []

In [29]:
import os
from PIL import Image

In [30]:
for path in os.listdir(Path):
    if path == 'scale_images.py':
        continue
    paths.append(os.path.join(Path,path))

In [31]:
mean = np.zeros((1024,1024))

In [32]:
for path in paths:
    if path  == '/datasets/ChestXray-NIHCC/images/scale_images.py':
        print('haha')

In [33]:
for i, path in enumerate(paths):
    
    meta = Image.open(path).convert(mode='L')
    image = np.array(meta) # a is readonly
    if image.shape == (1024,1024,4):
        print('4')
        continue
    mean += image
    
    if i % 5000 == 0:
        print(i)
        np.savetxt('tmp.txt',mean, fmt='%f') 
#     print(image.shape)
#     images.append(image)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000


In [34]:
i

112119

In [39]:
mean.shape

(1024, 1024)

In [38]:
np.savetxt('mean.txt',mean, fmt='%f')

In [26]:
len(images)

0

In [41]:
std_meta = np.zeros((1024,1024))

In [42]:
for i, path in enumerate(paths):
    
    meta = Image.open(path).convert(mode='L')
    image = np.array(meta) # a is readonly

    std_meta += (image-mean)**2
    
    if i % 5000 == 0:
        print(i)
        np.savetxt('std_meta.txt',std_meta, fmt='%f') 

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000


In [46]:
std = std_meta / 112119

In [50]:
np.savetxt('std.txt',std, fmt='%f') 